In [1]:
#!pip install datasets
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from datasets import load_dataset, load_from_disk
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans 
from sklearn.metrics import accuracy_score

/home/trbiv/OneDrive/School/Spring 2023/5995_applied_ml/project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Process Data

In [2]:
# dataset will be cached after loading the first time 

train_ds = load_dataset("mwritescode/slither-audited-smart-contracts", 'small-multilabel', split='train', ignore_verifications=True)
test_ds = load_dataset("mwritescode/slither-audited-smart-contracts", 'small-multilabel', split='test', ignore_verifications=True)

# for quick training (will cause bad output)
train_ds = train_ds.select(range(100))
test_ds = test_ds.select(range(50))

# cutting down dataset to speed up training, may cause less accurate model
#train_ds = train_ds.select(range(1000))
#test_ds = test_ds.select((range(200)))


# filter out empty bytecode 
train_ds = train_ds.filter(lambda elem: elem['bytecode'] != '0x')
test_ds = test_ds.filter(lambda elem: elem['bytecode'] != '0x')

# extract number of unique classes 


# generate RGB image 
def generate_RGB_image(example):
    # generate RGB image
    bytecode_string = example['bytecode']
    bytecode_hex = bytes.fromhex(bytecode_string[2:])
    image = np.frombuffer(bytecode_hex, dtype=np.uint8)
    length = int(np.ceil(len(image)/3))
    image = np.pad(image, pad_width=(0, length*3 - len(image)))
    image = image.reshape((-1, 3))
    sqrt_len = int(np.ceil(np.sqrt(image.shape[0])))
    image = np.pad(image,  pad_width=((0, sqrt_len**2 - image.shape[0]),(0,0)))
    image = image.reshape((sqrt_len, sqrt_len, 3))
    image = Image.fromarray(image)
    example['image'] = image

    return example

def generate_RGB_image_and_label(example):
    # generate RGB image
    bytecode_string = example['bytecode']
    bytecode_hex = bytes.fromhex(bytecode_string[2:])
    image = np.frombuffer(bytecode_hex, dtype=np.uint8)
    length = int(np.ceil(len(image)/3))
    image = np.pad(image, pad_width=(0, length*3 - len(image)))
    image = image.reshape((-1, 3))
    sqrt_len = int(np.ceil(np.sqrt(image.shape[0])))
    image = np.pad(image,  pad_width=((0, sqrt_len**2 - image.shape[0]),(0,0)))
    image = image.reshape((sqrt_len, sqrt_len, 3))
    image = Image.fromarray(image)
    example['image'] = image

    # 4 is considered safe
    example['label'] = 0.0 if 4 in example['slither'] else 1.0
    return example

# generate images, remove unneccessary columns 
train_ds = train_ds.map(generate_RGB_image, remove_columns=['address', 'source_code', 'bytecode', 'slither'])
test_ds = test_ds.map(generate_RGB_image_and_label, remove_columns=['address', 'source_code', 'bytecode', 'slither'])

# image after generating
train_ds[0]




/home/trbiv/OneDrive/School/Spring 2023/5995_applied_ml/project/.venv/lib/python3.10/site-packages/datasets/load.py:1744: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset slither-audited-smart-contracts (/home/trbiv/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/small-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)
Found cached dataset slither-audited-smart-contracts (/home/trbiv/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/small-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13)
Loading cached processed dataset at /home/trbiv/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/small-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f4664268

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=50x50>}

Apply augmentations

In [3]:

# pytorch expect RGB inputs in range [0, 1], normalize:
mean_rgb = [0.485, 0.456, 0.406]
std_rgb = [0.229, 0.224, 0.225]

# transform images
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_rgb, std=std_rgb)
])

# apply transformations
transformed_train_ds = train_ds.map(lambda elem: {'image': transform(elem['image'])})
transformed_test_ds = test_ds.map(lambda elem: {'image': transform(elem['image'])})

batch_size =  8

# convert to pytorch format 
transformed_train_ds = transformed_train_ds.with_format("torch")
transformed_test_ds = transformed_test_ds.with_format("torch")

# initialize dataloaders
train_dataloader = DataLoader(transformed_train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(transformed_test_ds, batch_size=batch_size, shuffle=False)

print(transformed_train_ds[2]['image'].shape)

Loading cached processed dataset at /home/trbiv/.cache/huggingface/datasets/mwritescode___slither-audited-smart-contracts/small-multilabel/1.1.0/4cf503b59ce9d3157914e47f6253de773b7ab828f46642685d4b470b88ca1f13/cache-3ebb13515260e0c0.arrow
                                                              

torch.Size([3, 128, 128])


In [4]:
class Unsupervised(nn.Module): 
    def __init__(self):
        super(Unsupervised, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 3, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(3)


    def forward(self, x):

        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))

        return x


# testig with dummy input 
model = Unsupervised()
input_size = (8, 3, 128, 128) 
dummy_input = torch.randn(*input_size)
output = model(dummy_input)
print("Output shape: ", output.shape)


Output shape:  torch.Size([8, 3, 128, 128])


In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = Unsupervised()
model = model.to(device)

# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
#num_epochs = 1 # because training takes so long 
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch_data in train_dataloader:
        inputs = batch_data["image"].to(device)

        # Forward pass
        #print("inputs: ", inputs.shape)
        outputs = model(inputs)

        # Backpropagation and optimization
        optimizer.zero_grad()
        #loss.backward()
        optimizer.step()

    # Validation 
    model.eval()
    total_loss = 0.0
    num_samples = 0
    embeddings = []
    true_labels = []
    with torch.no_grad():
        for batch_data in test_dataloader:
            inputs = batch_data["image"].to(device)

            outputs = model(inputs)

            batch_loss = torch.mean((inputs - outputs) ** 2) # mean squared error 
            total_loss =+ batch_loss.item() * inputs.size(0)
            num_samples += inputs.size(0)

            # apply k-means clustering on last epoch 
            if epoch == num_epochs - 1:
                outputs = outputs.reshape(-1, 3)
                embeddings.extend(outputs)
                true_labels.extend(batch_data["label"])


    if epoch == num_epochs - 1:
        # apply k-means clustering 
        num_clusters = 2 # safe or unsafe 
        kmeans = KMeans(n_clusters=num_clusters)
        embeddings = np.stack(embeddings)
        cluster_labels = kmeans.fit_predict(embeddings)
        #print("Cluster Labels Shape: ", cluster_labels.shape)

        # Aggregate cluster labels at the image level
        start_idx = 0
        image_labels = []
        for batch_data in test_dataloader:
            num_images = batch_data["image"].size(0)
            image_cluster_labels = cluster_labels[start_idx : start_idx + num_images]
            aggregated_label = np.bincount(image_cluster_labels).argmax()
            image_labels.extend([aggregated_label] * num_images)
            start_idx += num_images

        image_labels = np.array(image_labels)

        # calculate accuracy score 
        true_labels = np.array(true_labels)

        #print("True Labels Shape: ", true_labels.shape)
        accuracy = accuracy_score(true_labels, image_labels)
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss}, Accuracy Score: {accuracy}")
    else: 
        average_loss = total_loss / num_samples
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss}")


Epoch [1/5], Average Loss: 0.03701128035175557
Epoch [2/5], Average Loss: 0.036922406177131495
Epoch [3/5], Average Loss: 0.03659829314874143
Epoch [4/5], Average Loss: 0.036681459874522926


/home/trbiv/OneDrive/School/Spring 2023/5995_applied_ml/project/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch [5/5], Average Loss: 0.036681459874522926, Accuracy Score: 0.3520408163265306
